# CEE 609 - Term Project
## Adarsh Raghuram




## 1. Introduction

Heat stress is the leading cause of crop yield loss globally (Zhao et al., 2017, Schleussner et al., 2018). Exposure to high temperatures induces irreversible changes in crop physiology such as permanent leaf damage, accelerated aging and hastening of leaf senescence (Iqbal et al., 2017). The combined effect of such changes is reflected as reduction in yield. 

Numerous studies have examined the effects of high temperature stress on crop yields in the U.S., where Killing Degree Days (KDD) is often used as a metric to quantify sensitivity of crops to heat stress. For a given day, KDD is defined as the difference between the daily maximum temperature and a threshold temperature beyond which physiological effects of heat are observed in crops. For corn, the temperature threshold for KDD is defined as 29 degrees Celcius (Butler et al., 2018). While there are studies that independently examined the drought sensitivity of crop yields (Lobell et al., 2020), there are very few studies that examine the combined effect of temperature stress and soil moisture on crop yields. In the face of global challenges such as rising temperatures and food insecurity, tracking changes in sensitivity of corn to extreme temperatures is crucial for accurate estimation of future yields and identification of crop species and management practices resilient to climate change.This is particularly important considering the additional yield loss caused due to coupling between temperature and soil moisture (Lesk et al., 2021). This study examines the effect of temperature, soil moisture and their coupling on county level and state level corn yield anomalies in the rainfed states of the U.S.


### 1.1 Study Area
This project is centered over the eastern U.S., with specific focus on major corn growing regions (Fig 1). There are two primary reasons for choosing the eastern U.S. - 

- It includes the midwestern states which together produce 33% of global corn 
- Irrigation is known to reduce the impacts of heat stress crops. Since the majority of croplands in the eastern U.S. are rainfed, this naturally qualifies as control for irrigation

## 2. Data and Methods


### 2.1 Data Download and Processing

This study will use publicly available archival data on climate, soil moisture and crop yield, spanning six decades from 1960 to 2019. The ERA5 reanalysis data for temperature, precipitation and soil moisture (SM) data will be obtained from the Copernicus Climate Change Service database, maintained by the European Center for Medium Range Weather Forecasts (ECMWF), at a daily scale and a spatial resolution of 25 km. The python-based CDS API interface will be used for downloading this data (https://cds.climate.copernicus.eu/api-how-to). County-level corn yield will be obtained from the USDA database, accessible at https://data.nal.usda.gov/dataset/nass-quick-stats. Finally, to take into account the growing season of corn, planting and harvesting dates published in a 50 km gridded format by Sachs et.al3  will be obtained from the University of Wisconsin-Madison database accessible at https://sage.nelson.wisc.edu/data-and-models/datasets/crop-calendar-dataset/.Datasets.Methods. Refer to the 'data_process.ipynb' notebook for processing codes.

I accessed the USDA yield data directly from a web browser and the steps involved are described below:
- Go to the USDA Quickstats database page at https://quickstats.nass.usda.gov/
- Under the 'Select Commodity' section select the parameter mentioned for each category - Program: SURVEY -> Sector: CROPS -> Group: FIELD CROPS -> Commodity: CORN -> Data Item: CORN, GRAIN - YIELD, MEASURED IN BU/ACRE
- Under the 'Select Location" section select 'COUNTY' under Geographic Level
- Under the 'Select Time' section select years from 1960 to 1974
- After the data table is loaded click on 'spreadsheet' on the top right 
- Repeat the above steps with years 1975 to 1994 and again with years 1995 to 2020. This is necessary since not more than 50,000 records can be retrieved at once.

The yield data is then rasterized into a 50 km grid for matching the resolution of planting and harvesting dates using Climate Data Operators (CDO), a set of operators designed for processing and analyzing all forms of gridded data.

From the ERA5 server, daily maximum and minimum temperatures and daily soil moisture are downloaded using the API. After regridding all these to 50km, KDD and Growing Degree Days (GDD) are calculated at a daily scale using temperature and accumulated growing season KDD and GDD are calculated by adding daily numbers over the growing season (refer to the code for details). Here GDD accounts for the benefits of temperature increase on yield. Average growing season soil moisture is then calculated for every year, for each county.

All downloaded and processed data was then detrended to retain only the anomalies. This is necessary for examining the effect of a given parameter over a varying spaito-temporal range. 

### 2.2 Methods
A multilinear regression model is used for examining the effects of heat, soil moisture and their interactions on yield anomalies, iterating over each pixel. The form of the model is given below:

y<sub>it</sub> = B<sup>0</sup><sub>it</sub> + B<sup>k</sup><sub>it</sub>KDD<sub>it</sub> + B<sup>g</sup><sub>it</sub>GDD<sub>it</sub> + B<sup>s</sup><sub>it</sub>SM + B<sup>ks</sup><sub>it</sub>KDD<sub>it</sub> SM<sub>it</sub> + e<sub>it</sub>

Here y<sub>it</sub> is the yield anomaly in raster i and time t, B<sup>0</sup><sub>it</sub> is the space and time fixed effect, other Bs are coefficients representing yield sensitivities. The predictors used here are all detrended. The term B<sup>ks</sup><sub>it</sub>KDD<sub>it</sub> SM<sub>it</sub> captures the interaction between KDD and Soil Noisture. Overall, the model aims to estimate yield anomaly as a function of heat and soil moisture. This will help in understanding the spatial variations in sensitivity and identify patterns in the spread. Further, changes in senstivity will then be calculated over a moving window of 30-years, where the entire temporal range (1960-2019) will be divided into 31 windows of 30-years each (1960 to 1989, 1961 to 1990 and so on), and the trends in sensitivity for each coefficient will be calculated for each pixel. This will be useful for predicting future changes in sensitivities and estimating yield loss associated with heat and soil moisture effects.

In addition to the above form of the model, beacuse of the collinear relationship between heat (KDD) and soil moisture, regularization through ridge and elastic net regressions will also be performed using the cross-validation method. These methods help in introducing bias and decreasing the variance of the parameter estimates, to obtain a more practical and statistically significant estimates of sensitivities. Refer to the 'model_validation.ipynb' notebook for more details.

Finally, the rasterized model estimating county-scale effects will be scaled up to state level, and the multilinear regression and regularization models wil be tested.

### 3. Results
The coeffeicients from multilinear model iterated over pixels are not statistically significant over the entire area of interest (Fig 2). The primary reason for this is that there are few data points for each pixel (60) and there is some level of multicollinearity between the predictors. In order to account for the effects of collinearity, I calculated the coefficient of determination (R-squared) between KDD and SM to estimate the strenght of collinearity between the two variables. Very few pixels showed R-squared greater than 0.5, and this meant the variance inflation factor (VIF) was not greater than 10 for most of these pixels. As a result, the regularization techniques - ridge and elastic net - did not yield a consistant value for lambda when tested using cross-validation. Overall, no statistically significant effects of heat and soil moisture on yield anomalies were observed at a county level (pixel-level) and hence I decided to scale it up to state level.

The sensitivities were significant (p < 0.05) at state level and a north-south latitudinal gradient was observed (Fig 4), with northern states showing higher yield sensitivity to KDD. This could be due to the differences in crop varieties, management practices and coupled land-atmosphere processes (Lesk et al., 2021). The northern states appear to be more sensitive, and Iowa and Illinois being the top producers of corn in the U.S., this means a significant loss in yields. Therefore introcution of heat tolerant varieties of corn will be important for decreasing heat-stress caused yield loss in the midwestern states. 

Similarly, yield sensitivities to SM was also estimated, as shown in Fig 5. While states with positive yield anomalies explain the negative correlation between KDD and SM, those with negative yield anomalies could be because of changing precipitation patterns (this can be tested by running the model with only SM anomalies as the predictor and observing the trends in yield sensitivity to SM - I will be doing this next). Further, it could also be due to a latent process such as crop management practice, which the model is unable to capture. Overall, the unique spatial trends observed here highlighted the need for investigating the processes at a finer scale.

The next two plots show trends in yield senstivity to KDD and SM, estimated using a moving 30-year window. Fig 6 shows trends in yield sensitivity to KDDs, which is negative in most states. This means that for a given state with negative trend, a unit increase in KDD is causing more yield loss over time. Notably, these trends account for the effects of SM and GDD on yield and therefore reflect changes in either physiological response to KDD or interactions between KDD and SM. One way to test for changes in KDD-SM interaction would be by separating years with positive and negative SM anomalies and observing changes in correlation between KDD and SM. A higher KDD-SM relationship (negative) in dry years would mean high KDD occur in dry years and this could be driving the changes in yield sensitivity to KDD (this is my next step). Additionally, the effect of irrigation is not captured in the model - some states (North Dakota, South Dakota and Texas) have observed increasing trends in irrigation over years, which can offset the effects of KDD. Another factor that the model does not account for is the sensitivites of different growth phases (I will do this next) - there is evidence for varying KDD sensitivity of different growth phases of corn (Butler et al., 2018). 

The trends in sensitivity to soil moisture (Fig 7) seems to be the major driving factor behind variations in yield. Similar to the long term sensitivites in Fig 5, the negative trends observed in Iowa and Minnestoa could be reflective of the changes in KDD-SM relationships. Further, as noted above this could also be the result of changing precipitation patterns and intensity (Lesk et al., 2020). Further, crop management practices in these regions are different, and tillage type and irrigation are known to impact yields. In another project I'm currently wokring on, there is evidence that conservation tillage has a stabilizing effects of yields. Overall, a more detailed study would be necessary to clearly find out the underlying physical processes causing the observed patterns and trends in yield sensitivities.

Next, to account for collinearity, I tested the state-level relationships betwween KDD and SM. A plot showing the relationship for Iowa is shown in the below figure (Fig 8). Here, the ridge and elastic net regressions again faced problems with the strenght of collinearity - althoguh KDD and SM appear to be collinear, the R-squared is not high, making the VIF insignificant. Therefore the cross validation did not produce a stable optimum/minium lambda, resulting in uncertain estimates of model parameters.

## 4. Conclusion

The effects of KDD and SM on yield are not statistically significant at county levels and this maybe due to the coupled interactions of the terms, which the model is unable to discretize. Further, few data points available are insufficient to estimate the effects at a fine spatial scale. The latitudinal gradient in long term KDD sensitivity observed at state levels are mostly becuse of the changes in crop varieties and land-atmosphere coupling. The trends in sensitivity to soil moisture appears to be the major driving factor behind variations in yield anomalies and the influence of KDD is small compared to soil moistre. Trends in sensitivities offer a channel for further exploration, where the spatial patterns are not all explained by the existing literature. Additionally, dealing with partially collinear data sets still remains a problem, as in the case of this research. Although SM and KDD were obsevred to have some collinearity, the ridge and elastic net methods failed to find a static valvue for lamba to optimize the error. Overall, dealing with KDD and SM are both county and state levels need alternate statistical techniques to separate the effects of the two variables and understand the physical drivers of changes in yield anomalies.

## 5. Future Research Plans

- Changes in SM-KDD coupled interactions (separate dry and wet years and test trends in KDD sensitivities)
- Check for the variations in growth-phase specific sensitivities
- Use a different soil moisture data source (SAR)
- Check for the effects of tillage type 

## 6. References
- Butler, Ethan E., Nathaniel D. Mueller, and Peter Huybers. "Peculiarly pleasant weather for US maize." Proceedings of the National Academy of Sciences 115, no. 47 (2018): 11935-11940.
- Iqbal, M., N. Iqbal Raja, F. Yasmeen, M. Hussain, M. Ejaz, and M. A. Shah. "Impacts of heat stress on wheat: a critical review." Adv Crop Sci Technol 5, no. 1 (2017): 01-09.
- Lesk, Corey, Ethan Coffel, and Radley Horton. "Net benefits to US soy and maize yields from intensifying hourly rainfall." Nature Climate Change 10, no. 9 (2020): 819-822.
- Lesk, C., Coffel, E., Winter, J., Ray, D., Zscheischler, J., Seneviratne, S. I., & Horton, R. (2021). Stronger temperature–moisture couplings exacerbate the impact of climate warming on global crop yields. Nature food, 2(9), 683-691.
- Schleussner, Carl-Friedrich, Delphine Deryng, Christoph Müller, Joshua Elliott, Fahad Saeed, Christian Folberth, Wenfeng Liu et al. "Crop productivity changes in 1.5 C and 2 C worlds under climate sensitivity uncertainty." Environmental Research Letters 13, no. 6 (2018): 064007.
- Zhao, Chuang, Bing Liu, Shilong Piao, Xuhui Wang, David B. Lobell, Yao Huang, Mengtian Huang et al. "Temperature increase reduces global yields of major crops in four independent estimates." Proceedings of the National Academy of Sciences 114, no. 35 (2017): 9326-9331.


## 7. Reproducing this work

Note that this work is in progress and can be freely used and reproduced, in accordance with the MIT licence. Please email merge requests and other questions to araghura@syr.edu

The following are the required software and recommended hardware specifications:
### Software (required)
- R (version 4.1.1 or above) 
- R packages (install the latest version) - ncdf4, raster, repr, sf, ggplot2, glmnet
- Climate Data Operators (CDO) - install the latest version (check 'data_process.ipynb' for details)

### Hardware (recommended)
- Intel core-i7 11th Gen processor (8 core, 12 threads)
- 16 GB RAM


### Running the Codes
The 'data_process.ipynb' notebook provides a step-wise guide for downloading and processing data. After completing data processing, the following steps help in reproducing this work:

- All information about data sources and codes for data processing, model training and visualization are available in the 'Code' folder. Use the processed data (also available in the 'analysis_data' folder) as inputs for the models.
- Codes in the 'model_validation.ipynb' use data from the 'analysis_data' folder. Run the codes one by one and save the model outputs (all are available for reference in the 'model_outputs' folder) 
- Run the codes in 'visualization.ipynb' using files from the 'model_outputs' folder